In [1]:
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime
import pytz
import schedule

In [2]:

data_path="../../weatherman/"

In [3]:
def create_6days_list(no_of_days):
    a_day=86400
    day0 = datetime.strftime(datetime.utcnow(), "%s")
    past_6_days=[]

    for i in range(no_of_days):
        one_day=int(day0) - int(a_day*i)
        past_6_days.append(one_day)
    return(past_6_days)

past_6_days_list=create_6days_list(6)
past_6_days_list

[1635857817, 1635771417, 1635685017, 1635598617, 1635512217, 1635425817]

In [18]:
infer_dst = np.array([False] * entry.shape[0])  # all False -> every row considered DT, alternative is True to indicate DST. The array must correspond to the iloc of df.index
entry.index.tz_localize('Europe/London', ambiguous=infer_dst)  # no error

AttributeError: 'dict' object has no attribute 'shape'

In [4]:
api_key = "bc2d732248597a8725c1f5d51e5d3e8a"
lat=52.35
lon=-1.17
# lat=52.13
# lon=5.29
past_6_days = past_6_days_list

new_df=pd.DataFrame()
past_6_days_df=pd.DataFrame()

for day in past_6_days:
    url="https://api.openweathermap.org/data/2.5/onecall/timemachine?lat=%s&lon=%s&dt=%s&units=metric&appid=%s" % (lat, lon, day, api_key)
    response = requests.get(url)
    data = json.loads(response.text)
    hourly = data["hourly"]

    for entry in hourly:
        temp=pd.DataFrame.from_dict(pd.json_normalize(entry), orient='columns')

        # select only nec. columns
        columns=['dt','temp']
        temp=temp[columns]


        # transform dt column
        # temp['dt']=datetime.fromtimestamp(entry["dt"], pytz.timezone(data['timezone']))
        temp['dt']=datetime.fromtimestamp(entry["dt"], pytz.utc)
        new_df=new_df.append(temp,ignore_index=True)

    # concatenate each day in a new dataframe   
    past_6_days_df=pd.concat([past_6_days_df,new_df],axis=0, ignore_index=False).drop_duplicates()

past_6_days_df

In [31]:
# This is where the debug works
 
for entry in hourly:
    temp=pd.DataFrame.from_dict(pd.json_normalize(entry), orient='columns')

    # select only nec. columns
    columns=['dt','temp']
    temp=temp[columns]


    # transform dt column
    # temp['dt']=datetime.fromtimestamp(entry["dt"], pytz.timezone(data['timezone']))
    temp['dt']=datetime.fromtimestamp(entry["dt"], pytz.utc)
    new_df=new_df.append(temp,ignore_index=True)

# concatenate each day in a new dataframe   
past_6_days_df=pd.concat([past_6_days_df,new_df],axis=0, ignore_index=False).drop_duplicates()

past_6_days_df

,dt,temp
0,2021-10-28 00:00:00+00:00,13.17
1,2021-10-28 01:00:00+00:00,13.17
2,2021-10-28 02:00:00+00:00,13.17
3,2021-10-28 03:00:00+00:00,13.17
4,2021-10-28 04:00:00+00:00,13.17
5,2021-10-28 05:00:00+00:00,13.17
6,2021-10-28 06:00:00+00:00,13.17
7,2021-10-28 07:00:00+00:00,13.00
8,2021-10-28 08:00:00+00:00,13.25
9,2021-10-28 09:00:00+00:00,13.62


In [5]:
for entry in hourly:
        temp=pd.DataFrame.from_dict(pd.json_normalize(entry), orient='columns')

        # select only nec. columns
        columns=['dt','temp']
        temp=temp[columns]


        # # transform dt column
        # temp['dt']=datetime.fromtimestamp(entry["dt"], pytz.timezone(data['timezone']))
        # new_df=new_df.append(temp,ignore_index=True)

print(entry['dt'])
print(temp)
# print(temp['dt'])

# # Convert tz-naive to UTC
# tz = timezones.maybe_get_tz(tz)
# data = tzconversion.tz_localize_to_utc(data.view("i8"), tz, ambiguous=ambiguous)

1635462000
           dt   temp
0  1635462000  13.24


In [30]:
test=datetime.fromtimestamp(entry["dt"], pytz.timezone(data['timezone']))
# test
# test.strftime('%Y-%m-%d %H:%M:%S%')
# print(datetime.now(pytz.utc))
new_time=datetime.fromtimestamp(entry["dt"], pytz.utc)
new_time

# # tz = timezones.maybe_get_tz(tz)
# test2 = tzconversion.tz_localize_to_utc(test.view("i8"), tz, ambiguous=ambiguous)
# test2

# test.tz_localize('Europe/London')

datetime.datetime(2021, 10, 28, 23, 0, tzinfo=<UTC>)

In [21]:
api_key = "bc2d732248597a8725c1f5d51e5d3e8a"
# lat=52.35
# lon=-1.17
lat=52.13
lon=5.29
past_6_days = past_6_days_list

new_df=pd.DataFrame()
past_6_days_df=pd.DataFrame()

for day in past_6_days:
    url="https://api.openweathermap.org/data/2.5/onecall/timemachine?lat=%s&lon=%s&dt=%s&units=metric&appid=%s" % (lat, lon, day, api_key)
    response = requests.get(url)
    data = json.loads(response.text)
    hourly = data["hourly"]

    for entry in hourly:
        temp=pd.DataFrame.from_dict(pd.json_normalize(entry), orient='columns')

        # select only nec. columns
        columns=['dt','temp']
        temp=temp[columns]


        # transform dt column
        temp['dt']=datetime.fromtimestamp(entry["dt"], pytz.timezone(data['timezone']))
        new_df=new_df.append(temp,ignore_index=True)

    # concatenate each day in a new dataframe   
    past_6_days_df=pd.concat([past_6_days_df,new_df],axis=0, ignore_index=False).drop_duplicates()

past_6_days_df

AmbiguousTimeError: Cannot infer dst time from 2021-10-31 02:00:00, try using the 'ambiguous' argument

In [12]:
print(entry["dt"])
print(pytz.timezone(data['timezone']))

1635462000
Europe/London


In [19]:
datetime.fromtimestamp(entry["dt"], pytz.timezone(data['timezone']))

datetime.datetime(2021, 10, 29, 0, 0, tzinfo=<DstTzInfo 'Europe/London' BST+1:00:00 DST>)

In [ ]:
def get_historical_weather(lat,lon,api_key,past_6_days = past_6_days_list):

    new_df=pd.DataFrame()
    past_6_days_df=pd.DataFrame()

    for day in past_6_days:
        url="https://api.openweathermap.org/data/2.5/onecall/timemachine?lat=%s&lon=%s&dt=%s&units=metric&appid=%s" % (lat, lon, day, api_key)
        response = requests.get(url)
        data = json.loads(response.text)
        hourly = data["hourly"]

        for entry in hourly:
            temp=pd.DataFrame.from_dict(pd.json_normalize(entry), orient='columns')

            # select only nec. columns
            columns=['dt','temp']
            temp=temp[columns]

            # transform dt column
            temp['dt']=datetime.fromtimestamp(entry["dt"], pytz.timezone(data['timezone']))
            new_df=new_df.append(temp,ignore_index=True)

        # concatenate each day in a new dataframe   
        past_6_days_df=pd.concat([past_6_days_df,new_df],axis=0, ignore_index=False).drop_duplicates()

    return(past_6_days_df)

In [ ]:
# # Warning! Running this will overwrite current datasets

# germany=get_historical_weather(api_key = "bc2d732248597a8725c1f5d51e5d3e8a",lat=51.17,lon=10.45)
# germany.set_index(['dt']).to_csv(data_path + "germany_weather.csv")

# netherlands=get_historical_weather(api_key = "bc2d732248597a8725c1f5d51e5d3e8a",lat=52.13,lon=5.29)
# netherlands.set_index(['dt']).to_csv(data_path + "netherlands_weather.csv")

# uk=get_historical_weather(api_key = "bc2d732248597a8725c1f5d51e5d3e8a",lat=52.35,lon=-1.17)
# uk.set_index(['dt']).to_csv(data_path + "uk_weather.csv")

In [6]:
# Run this to append new entries

germany=get_historical_weather(api_key = "bc2d732248597a8725c1f5d51e5d3e8a",lat=51.17,lon=10.45)
pd.read_csv(data_path + "germany_weather.csv",index_col = False).set_index(['dt'])\
    .append(germany.set_index(['dt'])).drop_duplicates().to_csv(data_path + "germany_weather.csv")

netherlands=get_historical_weather(api_key = "bc2d732248597a8725c1f5d51e5d3e8a",lat=52.13,lon=5.29)
pd.read_csv(data_path + "netherlands_weather.csv",index_col = False).set_index(['dt'])\
    .append(netherlands.set_index(['dt'])).drop_duplicates().to_csv(data_path + "netherlands_weather.csv")

uk=get_historical_weather(api_key = "bc2d732248597a8725c1f5d51e5d3e8a",lat=52.35,lon=-1.17)
pd.read_csv(data_path + "uk_weather.csv",index_col = False).set_index(['dt'])\
    .append(uk.set_index(['dt'])).drop_duplicates().to_csv(data_path + "uk_weather.csv")

AmbiguousTimeError: Cannot infer dst time from 2021-10-31 02:00:00, try using the 'ambiguous' argument

In [ ]:

# new_df=pd.DataFrame()
# past_6_days_df=pd.DataFrame()

# for day in past_6_days:
#     url="https://api.openweathermap.org/data/2.5/onecall/timemachine?lat=%s&lon=%s&dt=%s&units=metric&appid=%s" % (lat, lon, day, api_key)
#     response = requests.get(url)
#     data = json.loads(response.text)
#     hourly = data["hourly"]

#     for entry in hourly:
#         # temp = pd.DataFrame(entry)
#         temp=pd.DataFrame.from_dict(pd.json_normalize(entry), orient='columns')

#         # select only nec. columns
#         columns=['dt','temp']
#         temp=temp[columns]

#         # transform dt column
#         temp['dt']=datetime.fromtimestamp(entry["dt"], pytz.timezone(data['timezone']))
#         new_df=new_df.append(temp,ignore_index=True)

#     # print(new_df)

#     # concatenate each day in a new dataframe   
#     past_6_days_df=pd.concat([past_6_days_df,new_df],axis=0, ignore_index=False).drop_duplicates()

# print(past_6_days_df)

In [ ]:
# new_df=pd.DataFrame()
# daily = data["daily"]
# for entry in daily:
#     dt = datetime.fromtimestamp(entry["dt"], pytz.timezone(data['timezone']))
#     temp = entry["temp"]
#     temp['dt']=dt
#     new_df=new_df.append(temp,ignore_index=True)
# new_df['timezone']=data['timezone']
# new_df

# this works for forecast

In [ ]:
# def get_weather_forecast(lat,lon,api_key):
    
#     new_df=pd.DataFrame()
#     url="https://api.openweathermap.org/data/2.5/onecall?lat=%s&lon=%s&exclude=current,minutely,hourly,alerts&units=metric&appid=%s" % (lat, lon, api_key)
#     response = requests.get(url)
#     data = json.loads(response.text)
    

#     daily = data["daily"]
#     for entry in daily:
#         dt = datetime.fromtimestamp(entry["dt"], pytz.timezone(data['timezone']))
#         temp = entry["temp"]
#         temp['dt']=dt
#         new_df=new_df.append(temp,ignore_index=True)
#     new_df['timezone']=data['timezone']
#     return(new_df)


In [ ]:
# germany=get_weather_forecast(api_key = "bc2d732248597a8725c1f5d51e5d3e8a",lat=51.17,lon=10.45)
# germany

# netherlands=get_weather_forecast(api_key = "3359ae5263476eb42c0c2bfa08ecc6c3",lat=52.13,lon=5.29)
# netherlands

# uk=get_weather_forecast(api_key = "3359ae5263476eb42c0c2bfa08ecc6c3",lat=52.35,lon=-1.17)
# uk

In [ ]:
# df=pd.DataFrame.from_dict(pd.json_normalize(data), orient='columns')